# Fazendo experimentos com `Executor.map`

Vamos invocar vários objetos de modo concorrente.

In [1]:
from time import sleep, strftime
from concurrent import futures

In [3]:
def display(*args):
    print(strftime('[%H:%M:%S]'), end=' ')
    print(*args)

def loiter(n):
    display('\t'* n + f"loiter({n}): fazendo nada por {n}s...")
    sleep(n)
    display('\t'* n + f"loiter({n}): finalizado.")
    return n * 10

In [11]:
def main(max_workers = 3, upper_bound_range=5):
    display('Script iniciado.')
    executor = futures.ThreadPoolExecutor(max_workers)
    results = executor.map(loiter, range(upper_bound_range))
    display('Resultados:', results)
    display('Aguardando os resultados individuais...')
    for i, result in enumerate(results):
        display(f'resultado {i}: {result}')

In [5]:
main()

[13:34:04] Script iniciado.
[13:34:04] loiter(0): fazendo nada por 0s...
[13:34:04] loiter(0): finalizado.
[13:34:04] 	loiter(1): fazendo nada por 1s...
[13:34:04] 		loiter(2): fazendo nada por 2s...
[13:34:04] 			loiter(3): fazendo nada por 3s...
[13:34:04] Resultados: <generator object Executor.map.<locals>.result_iterator at 0x105fe6020>
[13:34:04] Aguardando os resultados individuais...
[13:34:04] resultado 0: 0
[13:34:05] 	loiter(1): finalizado.
[13:34:05] 				loiter(4): fazendo nada por 4s...
[13:34:05] resultado 1: 10
[13:34:06] 		loiter(2): finalizado.
[13:34:06] resultado 2: 20
[13:34:07] 			loiter(3): finalizado.
[13:34:07] resultado 3: 30
[13:34:09] 				loiter(4): finalizado.
[13:34:09] resultado 4: 40


In [9]:
main(2)

[13:36:24] Script iniciado.
[13:36:24] loiter(0): fazendo nada por 0s...
[13:36:24] loiter(0): finalizado.
[13:36:24] 	loiter(1): fazendo nada por 1s...
[13:36:24] 		loiter(2): fazendo nada por 2s...
[13:36:24] Resultados: <generator object Executor.map.<locals>.result_iterator at 0x105fe75b0>
[13:36:24] Aguardando os resultados individuais...
[13:36:24] resultado 0: 0
[13:36:25] 	loiter(1): finalizado.
[13:36:25] 			loiter(3): fazendo nada por 3s...
[13:36:25] resultado 1: 10
[13:36:26] 		loiter(2): finalizado.
[13:36:26] 				loiter(4): fazendo nada por 4s...
[13:36:26] resultado 2: 20
[13:36:28] 			loiter(3): finalizado.
[13:36:28] resultado 3: 30
[13:36:30] 				loiter(4): finalizado.
[13:36:30] resultado 4: 40


In [10]:
main(4)

[13:37:16] Script iniciado.
[13:37:16] loiter(0): fazendo nada por 0s...
[13:37:16] loiter(0): finalizado.
[13:37:16] 	loiter(1): fazendo nada por 1s...
[13:37:16] 		loiter(2): fazendo nada por 2s...
[13:37:16] 			loiter(3): fazendo nada por 3s...
[13:37:16] 				loiter(4): fazendo nada por 4s...
[13:37:16] Resultados: <generator object Executor.map.<locals>.result_iterator at 0x105fe7b50>
[13:37:16] Aguardando os resultados individuais...
[13:37:16] resultado 0: 0
[13:37:17] 	loiter(1): finalizado.
[13:37:17] resultado 1: 10
[13:37:18] 		loiter(2): finalizado.
[13:37:18] resultado 2: 20
[13:37:19] 			loiter(3): finalizado.
[13:37:19] resultado 3: 30
[13:37:20] 				loiter(4): finalizado.
[13:37:20] resultado 4: 40


In [14]:
main(5,14)

[13:39:38] Script iniciado.
[13:39:38] loiter(0): fazendo nada por 0s...
[13:39:38] loiter(0): finalizado.
[13:39:38] 	loiter(1): fazendo nada por 1s...
[13:39:38] 		loiter(2): fazendo nada por 2s...
[13:39:38] 			loiter(3): fazendo nada por 3s...
[13:39:38] 				loiter(4): fazendo nada por 4s...
[13:39:38] 					loiter(5): fazendo nada por 5s...
[13:39:38] Resultados: <generator object Executor.map.<locals>.result_iterator at 0x11027c4f0>
[13:39:38] Aguardando os resultados individuais...
[13:39:38] resultado 0: 0
[13:39:39] 	loiter(1): finalizado.
[13:39:39] 						loiter(6): fazendo nada por 6s...
[13:39:39] resultado 1: 10
[13:39:40] 		loiter(2): finalizado.
[13:39:40] 							loiter(7): fazendo nada por 7s...
[13:39:40] resultado 2: 20
[13:39:41] 			loiter(3): finalizado.
[13:39:41] 								loiter(8): fazendo nada por 8s...
[13:39:41] resultado 3: 30
[13:39:42] 				loiter(4): finalizado.
[13:39:42] 									loiter(9): fazendo nada por 9s...
[13:39:42] resultado 4: 40
[13:39:43] 					l

Assim fica mais claro como ocorre a execução e como as threads são espertas.

Na execução acima, fica visível o término de uma thread e o início de outra, porque elas estão espaçadas em 5 linhas diferentes, então se a execução 1 terminou, a 6 acabou de iniciar.

## Problema

Os resultados são enviados na mesma ordem em que as chamadas foram iniciadas. Isso é ruim se você quiser que eles cheguem à medida que ficarem prontos, i.e. a ordem de recebimento deve ser o que ficar pronto primeiro.

Então, para enviar os resultados em ordem de prontidão, combinaremos `Executor.submit` para agendar um future e `futures.as_completed` para ir pegando os completos, mas isto será feito só depois.